In [40]:
import pandas as pd
import numpy as np
import scipy 
import matplotlib as plt
import seaborn as sns; sns.set()

In [41]:
df = pd.read_excel("SNMPdataset.xlsx")
df.drop(df.index[0],inplace=True)
df.head(5)


,SNMP Parameters,icmp_AddrMaskReps,icmp_InAddrMasks,icmp_InDestUnreachs,icmp_InEchosReps,icmp_InEchos,icmp_InErrors,icmp_InMsgs,icmp_InParmProbs,icmp_InRedirects,...,tcp_PassiveOpens,tcp_RetransSegs,tcp_RtoAlgorithm,tcp_RtoMax,tcp_RtoMin,udp_InDatagrams,udp_InErrors,udp_NoPorts,udp_OutDatagrams,ClassLabel
1,Mon Jul 31 15:49:58 UTC 2017 Counter32 value,0,0,0,16,0,0,17,0,0,...,1,0,1,120000,200,148,0,1,149,ICMP_Normal
2,Mon Jul 31 15:50:31 UTC 2017 Counter32 value,0,0,0,111,0,0,112,0,0,...,2,0,1,120000,200,445,0,1,446,ICMP_Normal
3,Mon Jul 31 15:51:10 UTC 2017 Counter32 value,0,0,0,229,0,0,230,0,0,...,2,0,1,120000,200,741,0,1,742,ICMP_Normal
4,Mon Jul 31 15:51:44 UTC 2017 Counter32 value,0,0,0,0,0,0,332,0,0,...,2,0,1,120000,200,1037,0,1,1038,ICMP_Normal
5,Mon Jul 31 15:52:24 UTC 2017 Counter32 value,0,0,0,442,0,0,443,0,0,...,2,0,1,120000,200,1333,0,1,1334,ICMP_Normal


In [44]:
df= df.replace(0,np.nan)
df=df.dropna(how='all',axis=1)
df=df.replace(np.nan,0)
del df["SNMP Parameters"]
df.head()


Index(['icmp_InEchosReps', 'icmp_InErrors', 'icmp_InMsgs', 'icmp_InTimeExcds',
       'icmp_InTimestamps', 'icmp_OutEchoReps', 'icmp_OutErrors',
       'icmp_OutTimeExcds', 'icmp_OutTimestamps', 'ip_DefaultTTL',
       'ip_Forwarding', 'ip_FragCreates', 'ip_FragOKs', 'ip_InAddrErrors',
       'ip_InDelivers', 'ip_InReceives', 'ip_OutRequests', 'ip_ReasmOKs',
       'ip_ReasmReqds', 'ip_ReasmTimeout', 'tcp_ActiveOpens',
       'tcp_CurrEstablish', 'tcp_EstabResets', 'tcp_InSegs', 'tcp_MaxConn',
       'tcp_OutRsts', 'tcp_OutSegs', 'tcp_PassiveOpens', 'tcp_RetransSegs',
       'tcp_RtoAlgorithm', 'tcp_RtoMax', 'tcp_RtoMin', 'udp_InDatagrams',
       'udp_NoPorts', 'udp_OutDatagrams', 'ClassLabel'],
      dtype='object')

In [ ]:
icmp_col = [col for col in df if col.startswith('icmp')]
icmp_col.append("ClassLabel")
tcp_col = [col for col in df if col.startswith('tcp')]
tcp_col.append("ClassLabel")

In [ ]:

df.columns
df_icmp=df[icmp_col]


In [ ]:
df_tcp=df[tcp_col]
df_tcp.head()

In [ ]:
df_icmp['ClassLabel'].unique()

In [ ]:
df_icmp=df_icmp.loc[df['ClassLabel'].isin(['ICMP_Normal','ICMP_Flood'])]
df_icmp.tail()

In [ ]:
from sklearn.cross_validation import train_test_split
y_icmp = df_icmp.pop('ClassLabel')
X_icmp = df_icmp
X_train_icmp,X_test_icmp,y_train_icmp,y_test_icmp = train_test_split(X_icmp,y_icmp,test_size=0.3)

In [ ]:
from sklearn import svm
y_train_icmp=np.array(y_train_icmp).reshape(-1,1)
y_test_icmp.shape

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,ExtraTreesClassifier 
from sklearn import preprocessing

In [ ]:
def models(X_train,X_test,y_train,y_test):
    names = ["Nearest Neighbors", "XGBoost","Decision Tree", "Random Forest","ExtraTreesClassifier","SVM"]
    classifiers = [KNeighborsClassifier(20),
                   xgb.XGBClassifier(learning_rate=0.01, max_depth=5, n_estimators=100),
                   DecisionTreeClassifier(max_depth=2),
                   RandomForestClassifier(max_depth=5, n_estimators=100,max_features=7),
                   ExtraTreesClassifier(max_depth=5,n_estimators=100,max_features=7),
                   svm.SVC()]
    for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print(name+' Score:',score)
        prediction1 = clf.predict(X_test)
        np.set_printoptions(precision=2)
    return 1
    

In [ ]:
names = ["Nearest Neighbors", "XGBoost",
         "Decision Tree", "Random Forest","ExtraTreesClassifier","SVM"]

classifiers = [
    KNeighborsClassifier(20),
    xgb.XGBClassifier(learning_rate=0.01, max_depth=5, n_estimators=100),
    DecisionTreeClassifier(max_depth=2),
    RandomForestClassifier(max_depth=5, n_estimators=100,max_features=7),
    ExtraTreesClassifier(max_depth=5,n_estimators=100,max_features=7),
svm.SVC()]

In [ ]:
for name, clf in zip(names, classifiers):
    clf.fit(X_train_icmp, y_train_icmp)
    score = clf.score(X_test_icmp, y_test_icmp)
    print(name+' Score:',score)
    prediction1 = clf.predict(X_test_icmp)
    np.set_printoptions(precision=2)

# tried on full data

In [ ]:
y = df.pop('ClassLabel')
X = df
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
y_train_icmp=np.array(y_train_icmp).reshape(-1,1)

In [ ]:
models(X_train,X_test,y_train,y_test)

In [ ]:
def sampler(df,category,class_labels):
    col = [col for col in df if col.startswith(category)]
    col.append("ClassLabel")
    df_tmp=df[col]
    df_tmp=df_tmp.loc[df['ClassLabel'].isin(class_labels)]
    y= df_tmp.pop('ClassLabel')
    X = df_tmp
    X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.3)
    return X_train,X_test,y_train,y_test 
    
    
    

In [ ]:
category=['icmp','tcp','udp',] 
for
X_train,X_test,y_train,y_test = sampler(df,'tcp',['TCPSYN_Normal', 'TCPSYNACK_Normal','TCPSYN_Flood','TCPSYNACK_Flood'])

